In [88]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from flask import Flask,request,jsonify

In [89]:
data = pd.read_csv("movies.csv")

In [90]:
data

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [91]:
data.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [92]:
data.isna().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [154]:
#preprocessing
df = data[['id','overview','title','genres','keywords','tagline','cast','director']]

In [155]:
df

,id,overview,title,genres,keywords,tagline,cast,director
0,19995,"In the 22nd century, a paraplegic Marine is di...",Avatar,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,285,"Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,206647,A cryptic message from Bond’s past sends him o...,Spectre,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,49026,Following the death of District Attorney Harve...,The Dark Knight Rises,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,49529,"John Carter is a war-weary, former military ca...",John Carter,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton
...,...,...,...,...,...,...,...,...
4798,9367,El Mariachi just wants to play his guitar and ...,El Mariachi,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,"He didn't come looking for trouble, but troubl...",Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez
4799,72766,A newlywed couple's honeymoon is upended by th...,Newlyweds,Comedy Romance,NaN,A newlywed couple's honeymoon is upended by th...,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns
4800,231617,"""Signed, Sealed, Delivered"" introduces a dedic...","Signed, Sealed, Delivered",Comedy Drama Romance TV Movie,date love at first sight narration investigati...,NaN,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith
4801,126186,When ambitious New York attorney Sam is sent t...,Shanghai Calling,NaN,NaN,A New Yorker in Shanghai,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia


In [156]:
df.size

38424

In [157]:
df.isna().sum()

id            0
overview      3
title         0
genres       28
keywords    412
tagline     844
cast         43
director     30
dtype: int64

In [158]:
df.dropna(inplace=True)

C:\Users\KIIT\AppData\Local\Temp\ipykernel_19432\1379821321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [159]:
df

,id,overview,title,genres,keywords,tagline,cast,director
0,19995,"In the 22nd century, a paraplegic Marine is di...",Avatar,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,285,"Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,206647,A cryptic message from Bond’s past sends him o...,Spectre,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,49026,Following the death of District Attorney Harve...,The Dark Knight Rises,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,49529,"John Carter is a war-weary, former military ca...",John Carter,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton
...,...,...,...,...,...,...,...,...
4792,36095,A wave of gruesome murders is sweeping Tokyo. ...,Cure,Crime Horror Mystery Thriller,japan prostitute hotel based on novel hallucin...,Madness. Terror. Murder.,Koji Yakusho Masato Hagiwara Tsuyoshi Ujiki An...,Kiyoshi Kurosawa
4793,182291,Isaac and Angel are two young Latinos involved...,On The Downlow,Drama,confession hazing gang member latino lgbt,Two gangs. One secret. One crossroad.,Tony Sancho Michael Cortez Donato Cruz Felipe ...,Tadeo Garcia
4795,124606,A young woman in L.A. is having a bad day: she...,Bang,Drama,gang audition police fake homeless actress,Sometimes you've got to break the rules,Darling Narita Peter Greene Michael Newland Er...,Ash Baron-Cohen
4796,14337,Friends/fledgling entrepreneurs invent a devic...,Primer,Science Fiction Drama Thriller,distrust garage identity crisis time travel ti...,What happens if it actually works?,Shane Carruth David Sullivan Casey Gooden Anan...,Shane Carruth


In [160]:
df.isna().sum()

id          0
overview    0
title       0
genres      0
keywords    0
tagline     0
cast        0
director    0
dtype: int64

In [161]:
df['overview'] = df['overview'].apply(lambda x: str(x).lower().replace(","," ").replace(":"," ").replace("!", " ").replace("...", " ").replace(".", " ").replace("?", " "))

C:\Users\KIIT\AppData\Local\Temp\ipykernel_19432\4206173029.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overview'] = df['overview'].apply(lambda x: str(x).lower().replace(","," ").replace(":"," ").replace("!", " ").replace("...", " ").replace(".", " ").replace("?", " "))


In [162]:
df['title'] = df['title'].apply(lambda x: str(x).lower().replace(","," ").replace(":"," ").replace("!", " ").replace("...", " ").replace(".", " ").replace("?", " "))

C:\Users\KIIT\AppData\Local\Temp\ipykernel_19432\3177045409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].apply(lambda x: str(x).lower().replace(","," ").replace(":"," ").replace("!", " ").replace("...", " ").replace(".", " ").replace("?", " "))


In [163]:
df['genres'] = df['genres'].apply(lambda x:str(x).lower())

C:\Users\KIIT\AppData\Local\Temp\ipykernel_19432\603840285.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres'] = df['genres'].apply(lambda x:str(x).lower())


In [164]:
df['keywords']=df['keywords'].apply(lambda x:str(x).lower())

C:\Users\KIIT\AppData\Local\Temp\ipykernel_19432\4242364902.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['keywords']=df['keywords'].apply(lambda x:str(x).lower())


In [165]:
df['tagline'] = df['tagline'].apply(lambda x: str(x).lower().replace(","," ").replace(":"," ").replace("!", " ").replace("...", " ").replace(".", " ").replace("?", " "))

C:\Users\KIIT\AppData\Local\Temp\ipykernel_19432\3431703867.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tagline'] = df['tagline'].apply(lambda x: str(x).lower().replace(","," ").replace(":"," ").replace("!", " ").replace("...", " ").replace(".", " ").replace("?", " "))


In [166]:
df['cast'] = df['cast'].apply(lambda x: str(x).lower().replace("\u00e9a", " "))

C:\Users\KIIT\AppData\Local\Temp\ipykernel_19432\1435819411.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cast'] = df['cast'].apply(lambda x: str(x).lower().replace("\u00e9a", " "))


In [167]:
df['director']=df['director'].apply(lambda x:str(x).lower())

C:\Users\KIIT\AppData\Local\Temp\ipykernel_19432\1764882868.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['director']=df['director'].apply(lambda x:str(x).lower())


In [168]:
df

,id,overview,title,genres,keywords,tagline,cast,director
0,19995,in the 22nd century a paraplegic marine is di...,avatar,action adventure fantasy science fiction,culture clash future space war space colony so...,enter the world of pandora,sam worthington zoe saldana sigourney weaver s...,james cameron
1,285,captain barbossa long believed to be dead ha...,pirates of the caribbean at world's end,adventure fantasy action,ocean drug abuse exotic island east india trad...,at the end of the world the adventure begins,johnny depp orlando bloom keira knightley stel...,gore verbinski
2,206647,a cryptic message from bond’s past sends him o...,spectre,action adventure crime,spy based on novel secret agent sequel mi6,a plan no one escapes,daniel craig christoph waltz l\u00e9a seydoux ...,sam mendes
3,49026,following the death of district attorney harve...,the dark knight rises,action crime drama thriller,dc comics crime fighter terrorist secret ident...,the legend ends,christian bale michael caine gary oldman anne ...,christopher nolan
4,49529,john carter is a war-weary former military ca...,john carter,action adventure science fiction,based on novel mars medallion space travel pri...,lost in our world found in another,taylor kitsch lynn collins samantha morton wil...,andrew stanton
...,...,...,...,...,...,...,...,...
4792,36095,a wave of gruesome murders is sweeping tokyo ...,cure,crime horror mystery thriller,japan prostitute hotel based on novel hallucin...,madness terror murder,koji yakusho masato hagiwara tsuyoshi ujiki an...,kiyoshi kurosawa
4793,182291,isaac and angel are two young latinos involved...,on the downlow,drama,confession hazing gang member latino lgbt,two gangs one secret one crossroad,tony sancho michael cortez donato cruz felipe ...,tadeo garcia
4795,124606,a young woman in l a is having a bad day she...,bang,drama,gang audition police fake homeless actress,sometimes you've got to break the rules,darling narita peter greene michael newland er...,ash baron-cohen
4796,14337,friends/fledgling entrepreneurs invent a devic...,primer,science fiction drama thriller,distrust garage identity crisis time travel ti...,what happens if it actually works,shane carruth david sullivan casey gooden anan...,shane carruth


In [169]:
#combining
df["combined"] = df["overview"]+ " "+df["genres"]+ " "+ df["keywords"]+" "+df["tagline"]+ " "+df["cast"]+" "+df["director"]

C:\Users\KIIT\AppData\Local\Temp\ipykernel_19432\3160511109.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["combined"] = df["overview"]+ " "+df["genres"]+ " "+ df["keywords"]+" "+df["tagline"]+ " "+df["cast"]+" "+df["director"]


In [170]:
df

,id,overview,title,genres,keywords,tagline,cast,director,combined
0,19995,in the 22nd century a paraplegic marine is di...,avatar,action adventure fantasy science fiction,culture clash future space war space colony so...,enter the world of pandora,sam worthington zoe saldana sigourney weaver s...,james cameron,in the 22nd century a paraplegic marine is di...
1,285,captain barbossa long believed to be dead ha...,pirates of the caribbean at world's end,adventure fantasy action,ocean drug abuse exotic island east india trad...,at the end of the world the adventure begins,johnny depp orlando bloom keira knightley stel...,gore verbinski,captain barbossa long believed to be dead ha...
2,206647,a cryptic message from bond’s past sends him o...,spectre,action adventure crime,spy based on novel secret agent sequel mi6,a plan no one escapes,daniel craig christoph waltz l\u00e9a seydoux ...,sam mendes,a cryptic message from bond’s past sends him o...
3,49026,following the death of district attorney harve...,the dark knight rises,action crime drama thriller,dc comics crime fighter terrorist secret ident...,the legend ends,christian bale michael caine gary oldman anne ...,christopher nolan,following the death of district attorney harve...
4,49529,john carter is a war-weary former military ca...,john carter,action adventure science fiction,based on novel mars medallion space travel pri...,lost in our world found in another,taylor kitsch lynn collins samantha morton wil...,andrew stanton,john carter is a war-weary former military ca...
...,...,...,...,...,...,...,...,...,...
4792,36095,a wave of gruesome murders is sweeping tokyo ...,cure,crime horror mystery thriller,japan prostitute hotel based on novel hallucin...,madness terror murder,koji yakusho masato hagiwara tsuyoshi ujiki an...,kiyoshi kurosawa,a wave of gruesome murders is sweeping tokyo ...
4793,182291,isaac and angel are two young latinos involved...,on the downlow,drama,confession hazing gang member latino lgbt,two gangs one secret one crossroad,tony sancho michael cortez donato cruz felipe ...,tadeo garcia,isaac and angel are two young latinos involved...
4795,124606,a young woman in l a is having a bad day she...,bang,drama,gang audition police fake homeless actress,sometimes you've got to break the rules,darling narita peter greene michael newland er...,ash baron-cohen,a young woman in l a is having a bad day she...
4796,14337,friends/fledgling entrepreneurs invent a devic...,primer,science fiction drama thriller,distrust garage identity crisis time travel ti...,what happens if it actually works,shane carruth david sullivan casey gooden anan...,shane carruth,friends/fledgling entrepreneurs invent a devic...


In [171]:
df_new=df.drop(columns = ['overview','genres','keywords','tagline','cast','director'])

In [172]:
df_new

,id,title,combined
0,19995,avatar,in the 22nd century a paraplegic marine is di...
1,285,pirates of the caribbean at world's end,captain barbossa long believed to be dead ha...
2,206647,spectre,a cryptic message from bond’s past sends him o...
3,49026,the dark knight rises,following the death of district attorney harve...
4,49529,john carter,john carter is a war-weary former military ca...
...,...,...,...
4792,36095,cure,a wave of gruesome murders is sweeping tokyo ...
4793,182291,on the downlow,isaac and angel are two young latinos involved...
4795,124606,bang,a young woman in l a is having a bad day she...
4796,14337,primer,friends/fledgling entrepreneurs invent a devic...


In [173]:
# encode the data
vectorizer = TfidfVectorizer(stop_words='english')
# encoded = vectorizer.fit(df["combined"])

In [174]:
encoded = vectorizer.fit_transform(df['combined']).toarray()

In [175]:
encoded

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [55]:
# #recommender function
# def recommend_movies(movie_name,top_rec):
#     movie_name = str(movie_name).lower().replace(",", " ").replace(":", " ")

#     #encode the input
#     inp_encoded = vectorizer.transform([movie_name])

#     similarity = cosine_similarity(encoded,inp_encoded)

#     #getting top rec to top indices
#     topindices = similarity.argsort()[::-1][:top_rec]
    
#     # return the recommended movies
#     x = data.iloc[topindices][["title"]]
#     print(type(x))
#     return x

In [85]:
# def recommend_movies(movie_name, top_rec):
#     movie_name = str(movie_name).lower().replace(",", " ").replace(":", " ")

#     # Encode the input
#     inp_encoded = vectorizer.transform([movie_name])
#     encoded_movies = vectorizer.transform(df["combined"])
#     similarity_scores = cosine_similarity(inp_encoded, encoded_movies)
#     # df["similarity"] = similarity_scores.flatten()
#     # df_sorted = df.sort_values("similarity", ascending=False)
#     # top_similar_movies = df_sorted[].head(top_rec)
#     # print(top_similar_movies)

#     top_indices = similarity_scores.argsort()[0][-top_rec:][::-1]
#     top_movies = data.iloc[top_indices]["title"] 
#     print(top_movies)




#     # similarity = cosine_similarity(encoded, inp_encoded.T)  # Transpose inp_encoded

#     # # Getting top recommendations by sorting similarity scores
#     # top_indices = similarity.argsort(axis=0)[::-1][:top_rec].flatten()

#     # # Return the recommended movies
#     # recommended_movies = df.iloc[top_indices]["title"]
#     # return recommended_movies

In [177]:
df[df['title'] == 'the lego movie'].index[0]


744

In [198]:
# def recommend_movies(movie_name,top_rec):
#     movie_name = str(movie_name).lower().replace(",", " ").replace(":", " ")
#     similarity = cosine_similarity(encoded)
#     # indices = df[df['title'] == movie_name].index[0]
#     # dist = sorted(list(enumerate(similarity[indices])),reverse=True,key = lambda x: x[1])

#     # for i in dist[:top_rec]:
#     #     print(df.iloc[i[0]].title)
#     # getting top rec to top indices
#     topindices = similarity.argsort()[:-1][:top_rec]
    
#     # return the recommended movies
#     x = df_new.iloc[topindices][["title"]]
#     # print(type(x))
#     # return x
#     print(x)


In [219]:
def recommend_movies(movie_name, top_rec):
    movie_name = str(movie_name).lower().replace(",", " ").replace(":", " ")
    similarity = cosine_similarity(encoded)
    
    # Find the index of the movie
    indices = df_new[df_new['title'] == movie_name].index[0]

    # Calculate the similarity scores for the given movie
    movie_scores = similarity[indices]

    # Sort the similarity scores and get the top recommendations
    top_indices = movie_scores.argsort()[::-1][:top_rec]

    # Return the recommended movies
    x = df_new.iloc[top_indices][['title']].to_dict(orient='records')
    # print(recommended_movies)
    return x

In [220]:
import json

In [223]:
def get_recommendation():
    movie_name = input("Enter the name of the movie: ")
    top_rec = 10
    result_dict = recommend_movies(movie_name, top_rec)
    with open('recommended_movies.json', 'w') as file:
        json.dump(result_dict, file)
    # Return the response as JSON
    # return jsonify(results=result_dict)

In [224]:
get_recommendation()